In [2]:
# Data packages
import os
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Math
import math
# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
# Import weights & Biases
import wandb

# From training.py 
from training import EarlyStopping, train_model, test_model

In [3]:
class ElasticWavePINN(nn.Module):
    def __init__(self):
        super(ElasticWavePINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 1)
        )

    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=1)
        return self.net(inputs)

In [4]:
def physics_informed_loss(model, x, t, mu, rho):
    u = model(x, t)
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u),
                              create_graph=True)[0]
    u_tt = torch.autograd.grad(u_t, t, grad_outputs=torch.ones_like(u_t),
                               create_graph=True)[0]
    u_xx = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                               create_graph=True, retain_graph=True)[0]
    u_xx = torch.autograd.grad(u_xx, x, grad_outputs=torch.ones_like(u_xx),
                               create_graph=True)[0]
    
    c_s = torch.sqrt(mu / rho)
    pde_loss = torch.mean((u_tt - c_s**2 * u_xx)**2)
    return pde_loss